## Get the probabilities of the csv

0. Create a new table with three columns: 
$$state, action, new\_state$$

<br/>

Now, we want to create a table for each action showing the probabilities of going from one state to another one.

In other words: $P_{a_t}(s_{t+1} | s_t)$

1. Create a dictionary where each key is an action and its value will be a Pandas DataFrame with the state and the new state.

2. Create a dictionary where each key is an action and its value will be a Pandas DataFrame with the state and the new state **probabilities**.

3. Save the probabilities in csv files.

### Load the CSV as a Pandas DataFrame

In [64]:
import pandas as pd
import numpy as np

In [65]:
# Note that the CSV is separated by ; (semicolon) instead of , (comma)
df = pd.read_csv('data.csv', sep=';')

In [66]:
df.head()

,Initial traffic level N,Initial traffic level E,Initial traffic level W,Green traffic light,Final traffic level N,Final traffic level E,Final traffic level W
0,High,High,Low,E,High,High,Low
1,High,High,Low,E,High,High,High
2,Low,High,Low,E,Low,High,High
3,High,High,High,N,High,High,High
4,High,High,High,W,High,High,Low


### 0. Create a new table with three columns: 
$$state, action → new \ state$$

In [67]:
# Get just the first letter of each element in the column
for column in df:
    df[column] = df[column].str[0]
df.head()

,Initial traffic level N,Initial traffic level E,Initial traffic level W,Green traffic light,Final traffic level N,Final traffic level E,Final traffic level W
0,H,H,L,E,H,H,L
1,H,H,L,E,H,H,H
2,L,H,L,E,L,H,H
3,H,H,H,N,H,H,H
4,H,H,H,W,H,H,L


In [68]:
# Concatenate the strings of the first three columns into one column
# called "state" and do the same for the new state
state = df[
    ['Initial traffic level N',
     'Initial traffic level E',
     'Initial traffic level W']
].apply(lambda x: ''.join(x), axis=1)

action = df['Green traffic light']

new_state = df[
    ['Final traffic level N',
     'Final traffic level E',
     'Final traffic level W']
].apply(lambda x: ''.join(x), axis=1)

columns = ['state', 'action', 'new_state']
mdp_df = pd.DataFrame({'state': state, 'action': action, 'new_state': new_state})
mdp_df.head()

,state,action,new_state
0,HHL,E,HHL
1,HHL,E,HHH
2,LHL,E,LHH
3,HHH,N,HHH
4,HHH,W,HHL


You can notice that states have only 7 possible values, while new_states have 8. That's because the state 'LLL' is only found on the new_state as it is the final state.

<br/>

Therefore, for each action we'll have a table with 7 rows and 8 columns.

In [69]:
states = mdp_df['state'].unique()
new_states = mdp_df['new_state'].unique()
print(f"States: {states}")
print(f"New States: {new_states}")

States: ['HHL' 'LHL' 'HHH' 'HLL' 'HLH' 'LLH' 'LHH']
New States: ['HHL' 'HHH' 'LHH' 'HLL' 'HLH' 'LLH' 'LHL' 'LLL']


### 1. Create a dictionary where each key is an action and its value will be a Pandas DataFrame with the state and the new state.

In [70]:
actions = mdp_df['action'].unique()
print(actions)

['E' 'N' 'W']


In [71]:
# Create a dictionary where each key is an action and its value will be a Pandas DataFrame with the state and the new state.
actions_dict = dict()

for action in actions:
    actions_dict[action] = mdp_df[mdp_df['action'] == action].reset_index(drop=True)
    actions_dict[action] = actions_dict[action].drop(columns=['action'])

print("DataFrame of action North:")
actions_dict['N'].head()

DataFrame of action North:


,state,new_state
0,HHH,HHH
1,HLL,HLL
2,HLL,HLL
3,HLH,HLH
4,HLH,HHH


### 2. Create a dictionary where each key is an action and its value will be a Pandas DataFrame with the state and the new state **probabilities**.

Example of $$P_N(s_{t+1} | s_t)$$

In [72]:
# Total count for each state ('HHL', 'LLH', 'HHH', etc.)
actions_total_count = actions_dict['N']['state'].value_counts()

count_df = pd.DataFrame(
    {'count': actions_dict['N'].groupby(['state', 'new_state']).size()}
)

print("Total count for the action North and state 'HHH':", end=' ')
print(actions_total_count['HHH'], end='\n\n')

print("Count of examples for the action North from the state 'HHH': ")
print(count_df.loc['HHH'], end='\n\n')

print("Probabilities of the action North from the state 'HHH': ")
hhh_probabilities = count_df.loc['HHH']['count'] / actions_total_count['HHH']
print(hhh_probabilities, end='\n\n')


Total count for the action North and state 'HHH': 417

Count of examples for the action North from the state 'HHH': 
           count
new_state       
HHH          262
LHH          155

Probabilities of the action North from the state 'HHH': 
new_state
HHH    0.628297
LHH    0.371703
Name: count, dtype: float64



In [73]:
probabilities_dict = dict()
states = mdp_df['state'].unique()
new_states = mdp_df['new_state'].unique()

for action in actions:
    actions_total_count = actions_dict[action]['state'].value_counts()

    count_df = pd.DataFrame(
        {'count': actions_dict[action].groupby(['state', 'new_state']).size()}
    )

    probabilities = pd.DataFrame(
        np.zeros((len(states), len(new_states))),
        index=states,
        columns=new_states
    )

    for state in states:
        probabilities.loc[state] = count_df.loc[state]['count'] / actions_total_count[state]

    probabilities_dict[action] = probabilities.fillna(0).round(3)


#### Print the probabilities:

$$P_W(s_{t+1} | s_t)$$

In [74]:
probabilities_dict['W']

,HHL,HHH,LHH,HLL,HLH,LLH,LHL,LLL
HHL,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00
LHL,0.313,0.000,0.000,0.000,0.000,0.000,0.687,0.00
HHH,0.320,0.680,0.000,0.000,0.000,0.000,0.000,0.00
HLL,0.340,0.000,0.000,0.660,0.000,0.000,0.000,0.00
HLH,0.121,0.227,0.000,0.203,0.449,0.000,0.000,0.00
LLH,0.046,0.063,0.135,0.076,0.145,0.315,0.081,0.14
LHH,0.115,0.228,0.462,0.000,0.000,0.000,0.195,0.00


$$P_N(s_{t+1} | s_t)$$

In [75]:
probabilities_dict['N']

,HHL,HHH,LHH,HLL,HLH,LLH,LHL,LLL
HHL,0.426,0.222,0.122,0.000,0.000,0.000,0.229,0.000
LHL,0.000,0.000,0.316,0.000,0.000,0.000,0.684,0.000
HHH,0.000,0.628,0.372,0.000,0.000,0.000,0.000,0.000
HLL,0.123,0.072,0.049,0.326,0.151,0.057,0.074,0.148
HLH,0.000,0.242,0.110,0.000,0.465,0.183,0.000,0.000
LLH,0.000,0.000,0.348,0.000,0.000,0.652,0.000,0.000
LHH,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000


$$P_E(s_{t+1} | s_t)$$

In [76]:
probabilities_dict['E']

,HHL,HHH,LHH,HLL,HLH,LLH,LHL,LLL
HHL,0.411,0.246,0.000,0.224,0.118,0.000,0.000,0.00
LHL,0.150,0.089,0.155,0.077,0.027,0.080,0.302,0.12
HHH,0.000,0.616,0.000,0.000,0.384,0.000,0.000,0.00
HLL,0.000,0.000,0.000,0.619,0.381,0.000,0.000,0.00
HLH,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.00
LLH,0.000,0.000,0.000,0.000,0.302,0.698,0.000,0.00
LHH,0.000,0.230,0.448,0.000,0.119,0.203,0.000,0.00


### 3. Save the probabilities in csv files.

In [77]:
probabilities_dict['W'].to_csv('probabilities_W.csv')
probabilities_dict['N'].to_csv('probabilities_N.csv')
probabilities_dict['E'].to_csv('probabilities_E.csv')

### 4. Search for probabilities

In [78]:
probabilities = pd.read_csv("probabilities_E.csv")
probabilities.columns = ['prev_state'] + list(probabilities.columns[1:])
probabilities.set_index('prev_state', inplace=True)

In [79]:
probabilities

,HHL,HHH,LHH,HLL,HLH,LLH,LHL,LLL
prev_state,,,,,,,,
HHL,0.411,0.246,0.000,0.224,0.118,0.000,0.000,0.00
LHL,0.150,0.089,0.155,0.077,0.027,0.080,0.302,0.12
HHH,0.000,0.616,0.000,0.000,0.384,0.000,0.000,0.00
HLL,0.000,0.000,0.000,0.619,0.381,0.000,0.000,0.00
HLH,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.00
LLH,0.000,0.000,0.000,0.000,0.302,0.698,0.000,0.00
LHH,0.000,0.230,0.448,0.000,0.119,0.203,0.000,0.00


In [80]:
probabilities.loc["LHL", "HLL"]

0.077